In [20]:
library("scPipe")

In [21]:
data.folder <- system.file("extdata", package = "scPipe", mustWork = TRUE)

In [22]:
data.folder

[1] "/SGRNJ01/Public/Software/conda_env/scpipe/lib/R/library/scPipe/extdata"

In [23]:
output_folder <- paste0("/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu", "/scPipeATACVignette")

In [24]:
output_folder

[1] "/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette"

In [25]:
# data fastq files
r1      <- file.path(data.folder, "small_chr21_R1.fastq.gz") 
r2      <- file.path(data.folder, "small_chr21_R3.fastq.gz") 

# barcodes in fastq format
barcode_fastq      <- file.path(data.folder, "small_chr21_R2.fastq.gz") 

# barcodes in .csv format
barcode_1000       <- file.path(data.folder, "chr21_modified_barcode_1000.csv")

In [26]:
sc_atac_trim_barcode (r1            = r1, 
                      r2            = r2, 
                      bc_file       = barcode_fastq, 
                      rmN           = FALSE,
                      rmlow         = FALSE,
                      output_folder = output_folder)

Saving the output at location: 
/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette



No valid_barcode_file provided; no barcode error correction will occur.
Total reads: 35000
Total reads removed due to N's in barcodes: 0
Total reads removed due to low quality barcodes: 0
Total barcodes provided in FASTQ file: 16567
time elapsed: 444 milliseconds



In [27]:
demux_r1        <- file.path(output_folder, "demux_completematch_small_chr21_R1.fastq.gz")
demux_r2        <- file.path(output_folder, "demux_completematch_small_chr21_R3.fastq.gz")
reference = file.path(data.folder, "small_chr21.fa")


aligned_bam <- sc_aligning(ref = reference, 
                R1 = demux_r1, 
                R2 = demux_r2, 
                nthreads  = 12,
                output_folder = output_folder)

ATAC-Seq mode is selected...

Genome index location not specified. Looking for the index in/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette

Genome index found in /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette...

No partial matches, checking for reads with non-matched barcodes.

No reads found with non-matching barcodes.

Outputted demultiplexing stats file to/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette/scPipe_atac_stats//demultiplexing_stats.csv


Output file name is not provided. Aligned reads are saved in /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette/demux_completematch_small_chr21_R1_aligned.bam




        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
       Rsubread 2.14.2

//================================= setting ==================================\\
||                                                                            ||
|| Function      : Read alignment (DNA-Seq)                                   ||
|| Input file 1  : demux_completematch_small_chr21_R1.fastq.gz                ||
|| Input file 2  : demux_completematch_small_chr21_R3.fastq.gz                ||
|| Output file   : demux_completematch_small_chr21_R1_aligned.bam (BAM),  ... ||
|| Index name    : genome_index                     

In [28]:
sorted_tagged_bam <- sc_atac_bam_tagging (inbam = aligned_bam, 
                       output_folder =  output_folder, 
                       bam_tags      = list(bc="CB", mb="OX"), 
                       nthreads      =  12)

Using default value for barcode length (bc_length = 16) 

Generating mapping statistics per barcode

Iterating through 5,000,000 reads at a time

chunk1

Completed generating mapping statistics per barcode, saved to /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette/scPipe_atac_stats/mapping_stats_per_barcode.csv


The output tagged and sorted BAM file was sent to /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette



In [29]:
removed <- sc_atac_remove_duplicates(sorted_tagged_bam,
                          output_folder = output_folder)
if (!isFALSE(removed))
  sorted_tagged_bam <- removed

samtools was located!

Sorting the BAM file by name

Running fixmate on the sorted BAM File

Sorting the BAM file by coordinates

Running samtools markdup now

Detected samtools with version greater or equal to 1.10. Producing duplicate removal stats file.

Indexing the BAM file

Cleaning up intermediary files

The output BAM file was sent to/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette



In [30]:
sc_atac_create_fragments(inbam = sorted_tagged_bam,
                         output_folder = output_folder)

Output folder name is: /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette
Output BED file: /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette/fragments.bed


In [31]:
# CHECK IF THE .narrowPeak file is small enough to include in the package,
# otherwise, we need to make this basilisk call work??
sc_atac_peak_calling(inbam = sorted_tagged_bam, 
                     ref = reference,
                     genome_size = NULL,
                     output_folder = output_folder)

Using genome size of1e+06



In [ ]:
?sc_atac_feature_counting

In [33]:
features          <- file.path(output_folder, "NA_peaks.narrowPeak")

sc_atac_feature_counting (fragment_file = file.path(output_folder, "fragments.bed"),
                          feature_input = features, 
                          bam_tags      = list(bc="CB", mb="OX"), 
                          feature_type  = "peak",
                          organism      = "hg38",
                          cell_calling  = "none",
                          min_uniq_frags = 0,
                          min_frac_peak = 0,
                          min_frac_promoter = 0,
                          yieldsize     = 1000000,
                          exclude_regions = TRUE,
                          output_folder = output_folder,
                          fix_chr       = "none",
                          create_report = TRUE
                          )

`peak`, `tss` or `gene` feature_type is selected for feature input

Creating Galignment object for the feature input ...

hg38is a recognized organism. Using annotation files in repository.

Generating TSS plot data

Generating feature-barcode fragment count matrix

Average no. of fragments per barcode: 1.35188866799205


Raw feature matrix generated: /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette/unfiltered_feature_matrix.rds

Calculating TSS enrichment scores

Generating QC metrics for cells


`none` function is used for cell calling ... 

nonewas not an implemented cell calling method

cell called and stored in /SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette

Cells with less than 200 counts are filtered out.

Number of cells to remove:1509

No cells were filtered out since otherwise there would be too few left.

features with less than 10 counts are filtered out.

Number of features to remove:4

No features were filtered out since otherwi

In [34]:
feature_matrix <- readRDS(file.path(output_folder, "unfiltered_feature_matrix.rds"))
dplyr::glimpse(feature_matrix)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:1550] 4 6 4 4 4 4 4 4 1 4 ...
  ..@ p       : int [1:1510] 0 1 2 3 4 5 6 7 8 9 ...
  ..@ Dim     : int [1:2] 11 1509
  ..@ Dimnames:List of 2
  .. ..$ feature: chr [1:11] "chr21:284484-285174" "chr21:287515-288092" "chr21:288148-288891" "chr21:289014-289231" ...
  .. ..$ barcode: chr [1:1509] "AAACGAACAGTAAGCG" "AAACGAACATGGATGG" "AAACGAATCGGACGAA" "AAACGAATCTGTGTGA" ...
  ..@ x       : num [1:1550] 1 1 1 1 1 2 1 1 1 1 ...
  ..@ factors : list()


In [ ]:
?sc_atac_create_sce

In [36]:
sce <- sc_atac_create_sce(input_folder = output_folder,
                   organism     = "hg38",
                   feature_type = "peak",
                   pheno_data   = NULL,
                   report       = TRUE)

SCE object is saved in:
	/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu/scPipeATACVignette/scPipe_atac_SCEobject.rds



In [15]:
?sc_atac_feature_counting

sc_atac_feature_counting {scPipe},R Documentation
fragment_file,The fragment file
feature_input,The feature input data e.g. the .narrowPeak file for peaks of a bed file format
bam_tags,The BAM tags
feature_type,The type of feature
organism,"The organism type (contains hg19, hg38, mm10)"
cell_calling,"The desired cell calling method; either cellranger, emptydrops or filter."
sample_name,The sample name to identify which is the data is analysed for.
genome_size,The size of the genome (used for the cellranger cell calling method)
promoters_file,The path of the promoter annotation file (if the specified organism isn't recognised).
tss_file,The path of the tss annotation file (if the specified organism isn't recognised).


In [ ]:
# data fastq files
r1      <- file.path(data.folder, "small_chr21_R1.fastq.gz") 
r2      <- file.path(data.folder, "small_chr21_R3.fastq.gz") 

# barcodes in fastq format
barcode_fastq      <- file.path(data.folder, "small_chr21_R2.fastq.gz") 

In [ ]:
?sc_atac_pipeline

In [ ]:
data.folder <- system.file("extdata", package = "scPipe", mustWork = TRUE)
output_folder <- paste0("/SGRNJ06/randd/USER/cjj/celedev/atac/20230607hs_mu", "/scPipeATACVignette")
sce <- sc_atac_pipeline(r1,
                        r2,
                        barcode_fastq,
                        organism = "hg38",
                        reference = file.path(data.folder, "small_chr21.fa"),
                        feature_type = "peak",
                        remove_duplicates = TRUE,
                        min_uniq_frags = 0, # set to 0 as the sample dataset only has a small number of reads
                        min_frac_peak = 0,
                        min_frac_promoter = 0,
                        report = TRUE,
                        output_folder = output_folder)

In [ ]:
# sample
library("scPipe")

In [ ]:
data.folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230811scpipe/test1/rawdata/"
output_folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230811scpipe/test1"
#reference <- "/SGRNJ06/randd/USER/cjj/cr-atac/refdata-cellranger-arc-GRCh38-2020-A-2.0.0/fasta/genome.fa"
reference <- "/SGRNJ01/Public/Software/conda_env/scpipe/lib/R/library/scPipe/extdata/small_chr21.fa"

In [ ]:
r1 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230811scpipe/test1/rawdata/A0727_1_3_MC_storage3_2W_T7_6515TESDSTween_TD612_C16_1.fq"
r2 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230811scpipe/test1/rawdata/A0727_1_3_MC_storage3_2W_T7_6515TESDSTween_TD612_C16_3.fq"

In [ ]:
# Aligning reads to a reference genome
aligned_bam <- sc_aligning(ref = reference, 
                R1 = r1, 
                R2 = r2, 
                nthreads  = 12,
                output_folder = output_folder)

In [ ]:
sorted_tagged_bam <- sc_atac_bam_tagging (inbam = aligned_bam, 
                       output_folder =  output_folder, 
                       bam_tags      = list(bc="CB", mb="OX"), 
                       nthreads      =  12)

In [ ]:
removed <- sc_atac_remove_duplicates(sorted_tagged_bam,
                          output_folder = output_folder)
if (!isFALSE(removed))
  sorted_tagged_bam <- removed

In [ ]:
sc_atac_create_fragments(inbam = sorted_tagged_bam,
                         output_folder = output_folder)

In [ ]:
# CHECK IF THE .narrowPeak file is small enough to include in the package,
# otherwise, we need to make this basilisk call work??
sc_atac_peak_calling(inbam = sorted_tagged_bam, 
                     ref = reference,
                     genome_size = NULL,
                     output_folder = output_folder)

In [ ]:
features          <- file.path(output_folder, "NA_peaks.narrowPeak")

sc_atac_feature_counting (fragment_file = file.path(output_folder, "fragments.bed"),
                          feature_input = features, 
                          bam_tags      = list(bc="CB", mb="OX"), 
                          feature_type  = "peak",
                          organism      = "hg38",
                          cell_calling  = "filter",
                          min_uniq_frags = 0,
                          min_frac_peak = 0,
                          min_frac_promoter = 0,
                          yieldsize     = 1000000,
                          exclude_regions = TRUE,
                          output_folder = output_folder,
                          fix_chr       = "none"
                          )

In [ ]:
feature_matrix <- readRDS(file.path(output_folder, "unfiltered_feature_matrix.rds"))
dplyr::glimpse(feature_matrix)

In [ ]:
sce <- sc_atac_create_sce(input_folder = output_folder,
                   organism     = "hg38",
                   feature_type = "peak",
                   pheno_data   = NULL,
                   report       = TRUE)

In [ ]:
# sample
library("scPipe")

In [ ]:
output_folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230811scpipe/test2"
reference <- "/SGRNJ06/randd/USER/cjj/soft/scpipe/ref/hg38.fa"

In [ ]:
sorted_tagged_bam <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230811scpipe/test2/A0727_1_3_MC_storage3_2W_T7_6515TESDSTween_TD612_C16_1_aligned_tagged_sorted.bam"

In [ ]:
sc_atac_create_fragments(inbam = sorted_tagged_bam,
                         output_folder = output_folder)

In [ ]:
# Peak calling
# CHECK IF THE .narrowPeak file is small enough to include in the package,
# otherwise, we need to make this basilisk call work??
sc_atac_peak_calling(inbam = sorted_tagged_bam,
                     ref = reference,
                     genome_size = NULL,
                     output_folder = output_folder)

In [ ]:
# Assigning reads to features and feature counting
features          <- file.path(output_folder, "NA_peaks.narrowPeak")
sc_atac_feature_counting (fragment_file = file.path(output_folder, "fragments.bed"),
                          feature_input = features,
                          bam_tags      = list(bc="CB", mb="OX"),
                          feature_type  = "peak",
                          organism      = "hg38",
                          cell_calling  = "filter",
                          min_uniq_frags = 0,
                          min_frac_peak = 0,
                          min_frac_promoter = 0,
                          yieldsize     = 1000000,
                          exclude_regions = TRUE,
                          output_folder = output_folder,
                          fix_chr       = "none"
                          )

In [ ]:
feature_matrix <- readRDS(file.path(output_folder, "unfiltered_feature_matrix.rds"))
dplyr::glimpse(feature_matrix)
sparseM <- readRDS(file.path(output_folder, "sparse_matrix.rds"))
dplyr::glimpse(sparseM)

In [ ]:
# Generating the Single-cell Experiment (SCE) object
sce <- sc_atac_create_sce(input_folder = output_folder,
                   organism     = "hg38",
                   feature_type = "peak",
                   pheno_data   = NULL,
                   report       = TRUE)

In [ ]:
data.folder <- system.file("extdata", package = "scPipe", mustWork = TRUE)

In [ ]:
data.folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/"
output_folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/scpipe"
reference <- "/SGRNJ06/randd/USER/cjj/soft/scpipe/ref/hg38.fa"
r1 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_1.fq"
r2 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_3.fq"

In [ ]:
aligned_bam <- sc_aligning(ref = reference,
                R1 = r1,
                R2 = r2,
                nthreads  = 12,
                output_folder = output_folder)

In [ ]:
?sorted_tagged_bam

In [ ]:
sorted_tagged_bam <- sc_atac_bam_tagging (inbam = aligned_bam, 
                       output_folder =  output_folder, 
                       bam_tags      = list(bc="CB", mb="OX"), 
                       nthreads      =  12)

In [ ]:
removed <- sc_atac_remove_duplicates(sorted_tagged_bam,
                          output_folder = output_folder)
if (!isFALSE(removed))
  sorted_tagged_bam <- removed

In [ ]:
sc_atac_create_fragments(inbam = sorted_tagged_bam,
                         output_folder = output_folder)

In [ ]:
sc_atac_peak_calling(inbam = sorted_tagged_bam, 
                     ref = reference,
                     genome_size = NULL,
                     output_folder = output_folder)

In [ ]:
sc_atac_feature_counting (fragment_file = file.path(output_folder, "fragments.bed"),
                          feature_input = reference,
                          bam_tags      = list(bc="CB", mb="OX"),
                          feature_type  = "genome_bin",
                          organism      = "hg38",
                          cell_calling  = FALSE,
                          yieldsize     = 1000000,
                          exclude_regions = TRUE,
                          output_folder = output_folder,
                          fix_chr       = "none"
                          )

In [ ]:
?sc_atac_feature_counting

In [ ]:
system.file("extdata", package = "scPipe", mustWork = TRUE)

In [ ]:
library("scPipe")

In [ ]:
output_folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/scpipe"
reference <- "/SGRNJ06/randd/USER/cjj/soft/scpipe/ref/hg38.fa"
r1 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_1.fq"
r2 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_3.fq"
bc_fastq = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_2.fq"

In [ ]:
output_folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/scpipe"
sce <- sc_atac_pipeline(r1,
                        r2,
                        bc_fastq,
                        organism = "hg38",
                        reference = reference,
                        feature_type = "peak",
                        remove_duplicates = FALSE,
                        min_uniq_frags = 0, # set to 0 as the sample dataset only has a small number of reads
                        min_frac_peak = 0,
                        min_frac_promoter = 0,
                        output_folder = output_folder,
                        report       = TRUE)

In [ ]:
?sc_atac_pipeline

In [ ]:
library(scPipe)

In [ ]:
data.folder

In [ ]:
output_folder <- "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/test1"
reference <- "/SGRNJ06/randd/USER/cjj/soft/scpipe/ref/latest/hg38.fa"
#reference <- "/SGRNJ01/Public/Software/conda_env/scpipe/lib/R/library/scPipe/extdata/small_chr21.fa"
#r1 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_1.fq.gz"
#r2 = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_3.fq.gz"
#bc_fastq = "/SGRNJ06/randd/USER/cjj/celedev/atac/20230815ds/A0322_1_H_prostate_959595_T7/01.barcode/A0322_1_H_prostate_959595_T7_2.fq.gz"

r1 = "/SGRNJ01/Public/Software/conda_env/scpipe/lib/R/library/scPipe/extdata/small_chr21_R1.fastq.gz"
r2 = "/SGRNJ01/Public/Software/conda_env/scpipe/lib/R/library/scPipe/extdata/small_chr21_R3.fastq.gz"
bc_fastq = "/SGRNJ01/Public/Software/conda_env/scpipe/lib/R/library/scPipe/extdata/small_chr21_R2.fastq.gz"

In [ ]:
sc_atac_trim_barcode (r1            = r1, 
                      r2            = r2, 
                      bc_file       = bc_fastq, 
                      rmN           = FALSE,
                      rmlow         = FALSE,
                      output_folder = output_folder)

In [ ]:
demux_r1        <- file.path(output_folder, "demux_completematch_small_chr21_R1.fastq.gz")
demux_r2        <- file.path(output_folder, "demux_completematch_small_chr21_R3.fastq.gz")

aligned_bam <- sc_aligning(ref = reference, 
                R1 = demux_r1, 
                R2 = demux_r2, 
                nthreads  = 12,
                output_folder = output_folder)

In [ ]:
sorted_tagged_bam <- sc_atac_bam_tagging (inbam = aligned_bam, 
                       output_folder =  output_folder, 
                       bam_tags      = list(bc="CB", mb="OX"), 
                       nthreads      =  12)

In [ ]:
removed <- sc_atac_remove_duplicates(sorted_tagged_bam,
                          output_folder = output_folder,)
if (!isFALSE(removed))
  sorted_tagged_bam <- removed

In [ ]:
sc_atac_create_fragments(inbam = sorted_tagged_bam,
                         output_folder = output_folder)

In [ ]:
sc_atac_peak_calling(inbam = sorted_tagged_bam, 
                     ref = reference,
                     genome_size = NULL,
                     output_folder = output_folder)

In [ ]:
features          <- file.path(output_folder, "NA_peaks.narrowPeak")
sc_atac_feature_counting (fragment_file = file.path(output_folder, "fragments.bed"),
                          feature_input = features, 
                          bam_tags      = list(bc="CB", mb="OX"), 
                          feature_type  = "peak",
                          organism      = "hg38",
                          cell_calling  = "filter",
                          min_uniq_frags = 0,
                          min_frac_peak = 0,
                          min_frac_promoter = 0,
                          yieldsize     = 1000000,
                          exclude_regions = TRUE,
                          output_folder = output_folder,
                          fix_chr       = "none"
                          )

In [ ]:
feature_matrix <- readRDS(file.path(output_folder, "unfiltered_feature_matrix.rds"))
dplyr::glimpse(feature_matrix)

In [ ]:
sparseM <- readRDS(file.path(output_folder, "sparse_matrix.rds"))
dplyr::glimpse(sparseM)

In [ ]:
sce <- sc_atac_create_sce(input_folder = output_folder,
                   organism     = "hg38",
                   feature_type = "peak",
                   pheno_data   = NULL,
                   report       = TRUE)

In [ ]:
?sc_atac_trim_barcode